In [ ]:
# from ipyleaflet import Map, Marker

# # Create a map centered at a specific location (latitude, longitude)
# center = (45.5236, -122.6750)  # Coordinates for Portland, Oregon
# mymap = Map(center=center, zoom=10)

# # Create a marker and add it to the map
# marker = Marker(location=center, draggable=False)
# mymap.add_layer(marker)

# # Display the map
# mymap

In [1]:
import geemap
import ee
import os

In [2]:
email = "mt-hvi-zonalstats-service-acco@mt-hvi-zonalstats.iam.gserviceaccount.com"
key_file = "/Users/natebender/Desktop/repo/mt-hvi-zonalstats-6da42ca28c80.json"

# Authenticate and initialize
credentials = ee.ServiceAccountCredentials(email=email, key_file=key_file)
ee.Initialize(credentials)

In [ ]:


# Define the area of interest (Montana)
aoi = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq("STATEFP", "30")).geometry()


In [ ]:
# Function to apply scaling factors
def apply_scale_factors(image):
    thermal_bands = image.select('ST_B10').multiply(0.00341802).add(149.0)
    return image.addBands(thermal_bands.rename('ST_B10_scaled'), overwrite=True)

# Function to mask clouds
def cloud_mask(image):
    cloud_shadow_bitmask = (1 << 3)
    cloud_bitmask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bitmask).eq(0).And(qa.bitwiseAnd(cloud_bitmask).eq(0))
    return image.updateMask(mask)

In [ ]:
# Get summer temperatures
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2021-06-01', '2021-08-31') \
    .filter(ee.Filter.lte('CLOUD_COVER', 10)) \
    .map(apply_scale_factors) \
    .map(cloud_mask)

# Median image of the collection
median_image = collection.median().clip(aoi)

In [ ]:
fahrenheit_image = median_image.select('ST_B10_scaled').subtract(273.15).multiply(9/5).add(32)

In [ ]:
# Create a geemap map
Map = geemap.Map(center=(47, -110), zoom=6)

# Define visualization parameters
vis_params = {
    'min': 50,    # Set minimum temperature value for visualization
    'max': 100,   # Set maximum temperature value for visualization
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
}



In [ ]:
# Add the image layer to the map with a short description
Map.addLayer(median_image, vis_params, "LST2021")

# Display the map
Map

In [ ]:
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017');
zambia = countries.filter(ee.Filter.eq("country_na","Zambia"));

usa = countries.filter(ee.Filter.eq("country_na","United States"));

#aoi = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq("STATEFP", "30"));
aoi = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("STUSPS", "MT"))


In [ ]:
#dataset = ee.ImageCollection('LANDSAT/LC08/CO1/T1_TOA').filterDate('2017-01-01', '2017-12-31').filterBounds(Zambia)
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2017-01-01', '2017-01-15').filterBounds(aoi)

In [ ]:
truecolor432 = dataset.select(['ST_B10'])
truecolorvis = {
    'min': 0.0,
    'max': 0.4,
    }

In [ ]:
Map = geemap.Map(center=(40, -100), zoom=4, lite_mode=True)
Map.addLayer(ee.Image().paint(usa, 0, 2), {}, 'Zambia')
Map.addLayer(truecolor432.median(), truecolorvis, "test")

Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

In [3]:
# Define the area of interest (aoi) as the boundary of Montana.
aoi = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("STUSPS", "MT"))

# Function to apply scale factors to the Landsat data
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_band = image.select('ST_B10').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_band, None, True)

# Function to mask clouds based on the pixel_qa band of Landsat SR data.
def cloud_mask(image):
    qa = image.select('QA_PIXEL')
    cloud = qa.bitwiseAnd(1 << 3).neq(0)
    return image.updateMask(cloud.Not())

# Get the Landsat 8 image collection.
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2021-06-01', '2021-09-15') \
    .filter(ee.Filter.lte('CLOUD_COVER', 10)) \
    .map(apply_scale_factors) \
    .map(cloud_mask)

# Select the thermal band and convert from Kelvin to Fahrenheit
median = collection.select(['ST_B10']).median().clip(aoi)
fahrenheit_image = median.subtract(273.15).multiply(9 / 5).add(32)

# Visualization parameters for the temperature layer
temp_vis = {
    'min': 40,    # Set minimum temperature value for visualization
    'max': 110,
    'palette': ['000004', '1b0c41', '4a0c6b', '781c6d', 'a52c60', 'cf4446', 'ed6925', 'fb9b06', 'f7d13d', 'fcffa4']
}


In [5]:
# Create a map
Map = geemap.Map(center=(40, -100), zoom=4, lite_mode=True)


# Add the temperature layer to the map
Map.addLayer(fahrenheit_image, temp_vis, 'Summer Temperatures')
Map.addLayer(ee.Image().paint(aoi, 0, 2), {}, 'Montana')

# Define the legend dictionary
legend_dict = {
    '40-50°F': '#000004',
    '50-60°F': '#1b0c41',
    '60-70°F': '#4a0c6b',
    '70-80°F': '#781c6d',
    '80-90°F': '#a52c60',
    '90-100°F': '#cf4446',
    '100-110°F': '#ed6925',
    '110+°F': '#fcffa4'
}

# Add the legend to the map

# Display the map with controls
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map.add_legend(title="Temperature (°F)", legend_dict=legend_dict)

Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…